In [ ]:
from scraper import Scraper
import pandas as pd
import httpx
from bs4 import BeautifulSoup
from random import random, randint
import time
import pandas as pd
from datetime import date

In [ ]:
s = Scraper()

In [ ]:
cities = ['amsterdam']
#sites = ['pararius','funda','kamernet']
#sites = ['kamernet']
sites = ['pararius']
postTypes = ['Buy','Rent']

In [ ]:
s.run(cities,sites,postTypes,10000)

In [ ]:
s.properties

In [ ]:
update_df = history.set_index('url')[['first_scrape_date', 'last_scrape_date']].join(
    s.properties.set_index('url')['scrape_date'], 
    how='outer',
    rsuffix=''
)

In [ ]:
new_properties = s.properties[s.properties['url'].isin(update_df[update_df['first_scrape_date'].isna()].index)].copy()

In [ ]:
new_properties['first_scrape_date'] = new_properties['scrape_date']
new_properties['last_scrape_date'] = new_properties['scrape_date']
new_properties = new_properties.drop(columns=['scrape_date'])

In [ ]:
properties_to_update = update_df[((~update_df['first_scrape_date'].isna())&
                                  (~update_df['scrape_date'].isna()))]

In [ ]:
history = history.set_index('url')
history.loc[history.index.isin(properties_to_update.index),'last_scrape_date'] = properties_to_update['scrape_date']
history = history.reset_index()

In [ ]:
history = pd.concat([history, new_properties])

In [ ]:
history.to_csv('data/history_20241207.csv', index=False)

In [ ]:
history = pd.read_csv('data/history_20241207.csv')

In [ ]:
postcode_coord = pd.read_csv('data/postcode_coordinates.csv')

In [ ]:
history['latitude'] = history['post_code'].map(postcode_coord.set_index('post_code')['latitude'].to_dict())
history['longitude'] = history['post_code'].map(postcode_coord.set_index('post_code')['longitude'].to_dict())

In [ ]:
history = history.rename(columns={'postType': 'post_type', 'post_code': 'postcode'})

In [ ]:
history.to_csv('data/history_20241207.csv', index=False)

In [ ]:
history = history[
    ['source', 'first_scrape_date', 'last_scrape_date', 'url', 'post_type',
    'city', 'location', 'title', 'postcode', 'price', 'price_type', 'surface',
    'surface_unit', 'rooms', 'furnished', 'latitude', 'longitude']
]

In [ ]:
history = history[~history['price'].isna()]

In [ ]:
history['price'] = history['price'].astype(int)

In [ ]:
history.to_csv('data/history_20241207_nohead.csv', index=False, header=False)